In [1]:
!pip install -qU kaggle

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.6/84.6 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [5]:
import os

kaggle_dir = '/root/.kaggle'
os.makedirs(kaggle_dir, exist_ok=True)

!mv kaggle.json {kaggle_dir}

In [6]:
!chmod 600 {kaggle_dir}/kaggle.json

In [7]:
import kaggle

In [8]:
!kaggle datasets download ryanrudes/yttts-speech

100% 9.91G/9.91G [05:53<00:00, 36.8MB/s]
100% 9.91G/9.91G [05:53<00:00, 30.1MB/s]


In [9]:
!unzip yttts-speech.zip

Streaming output truncated to the last 5000 lines.
  inflating: data/yexR53My2O4/00:03:26,459-00:03:28,290/audio.wav  
  inflating: data/yexR53My2O4/00:03:26,459-00:03:28,290/subtitles.txt  
  inflating: data/yexR53My2O4/00:03:28,290-00:03:30,030/audio.wav  
  inflating: data/yexR53My2O4/00:03:28,290-00:03:30,030/subtitles.txt  
  inflating: data/yexR53My2O4/00:03:30,030-00:03:33,300/audio.wav  
  inflating: data/yexR53My2O4/00:03:30,030-00:03:33,300/subtitles.txt  
  inflating: data/yexR53My2O4/00:03:33,300-00:03:36,510/audio.wav  
  inflating: data/yexR53My2O4/00:03:33,300-00:03:36,510/subtitles.txt  
  inflating: data/yexR53My2O4/00:03:36,510-00:03:39,150/audio.wav  
  inflating: data/yexR53My2O4/00:03:36,510-00:03:39,150/subtitles.txt  
  inflating: data/yexR53My2O4/00:03:39,150-00:03:41,780/audio.wav  
  inflating: data/yexR53My2O4/00:03:39,150-00:03:41,780/subtitles.txt  
  inflating: data/yexR53My2O4/00:03:41,780-00:03:45,660/audio.wav  
  inflating: data/yexR53My2O4/00:03:41,78

In [10]:
!pip install -qU bs4 tqdm datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.0 MB/s eta 0:00:00


In [11]:
import os
import time
import requests
from tqdm.auto import tqdm
from bs4 import BeautifulSoup

In [14]:
video_id = os.listdir("data")
video_id[:4]

['YqvhDPd1UEw', 'etsayyDGiO0', '7Q2JhZxNPow', 'hDQNCWR3HLQ']

In [15]:
splits = sorted(os.listdir(f"data/{video_id[2]}"))
splits[:5]

['00:00:00,030-00:00:02,280',
 '00:00:02,280-00:00:04,799',
 '00:00:04,799-00:00:10,019',
 '00:00:10,019-00:00:11,940',
 '00:00:11,940-00:00:16,500']

In [18]:
with open(f"data/{video_id[2]}/{splits[0]}/subtitles.txt") as f:
    text = f.read()
print(text)

next our each other I renege we'll talk


In [22]:
documents = []
for video_id in tqdm(video_ids):
    splits = sorted(os.listdir(f"data/{video_id}"))
    # we start at 00:00:00
    start_timestamp = "00:00:00"
    passage = ""
    for i, s in enumerate(splits):
        with open(f"data/{video_id}/{s}/subtitles.txt") as f:
            # append tect to current chunk
            out = f.read()
            passage += " " + out
        # average sentence length is 75-100 characters so we will cut off
        # around 3-4 sentences
        if len(passage) > 360:
            # now we've hit the needed length create a record
            # extract the end timestamp from the filename
            end_timestamp = s.split("-")[1].split(",")[0]
            # extract string timestamps to actual datetime objects
            start = time.strptime(start_timestamp,"%H:%M:%S")
            end = time.strptime(end_timestamp,"%H:%M:%S")
            # now we extract the second/minute/hour values and convert
            # to total number of seconds
            start_second = start.tm_sec + start.tm_min*60 + start.tm_hour*3600
            end_second = end.tm_sec + end.tm_min*60 + end.tm_hour*3600
            # save this to the documents list
            documents.append({
                "video_id": video_id,
                "text": passage,
                "start_second": start_second,
                "end_second": end_second,
                "url": f"https://www.youtube.com/watch?v={video_id}&t={start_second}s",
            })
            # now we update the start_timestamp for the next chunk
            start_timestamp = end_timestamp
            # refresh passage
            passage = ""

  0%|          | 0/127 [00:00<?, ?it/s]

In [23]:
documents[:3]

[{'video_id': 'YqvhDPd1UEw',
  'text': " hi everyone welcome to lecture 12 of deep unsupervised learning spring 2020 today we'll cover representation learning in reinforcement learning first before I start I wanna give a big thank you to many colleagues and friends who have contributed to this lecture through sharing their insights illustrations slides and videos that all very directly contributed to well I'll be sharing in",
  'start_second': 0,
  'end_second': 28,
  'url': 'https://www.youtube.com/watch?v=YqvhDPd1UEw&t=0s'},
 {'video_id': 'YqvhDPd1UEw',
  'text': " this lecture here today thank you so Terry what class has been about unsupervised learning today we're actually going to look at how one surprise learning and reinforcement can be brought together in a way that inspires learning to make reinforced learning more efficient but we haven't covered reinforcement learning yet in this class and so what I'm first gonna",
  'start_second': 28,
  'end_second': 52,
  'url': 'https://

In [24]:
import lxml

metadata = {}
for _id in tqdm(video_ids):
    r = requests.get(f"https://www.youtube.com/watch?v={_id}")
    soup = BeautifulSoup(r.content, 'lxml')  # lxml package is used here
    try:
        title = soup.find("meta", property="og:title").get("content")
        thumbnail = soup.find("meta", property="og:image").get("content")
        metadata[_id] = {"title": title, "thumbnail": thumbnail}
    except Exception as e:
        print(e)
        print(_id)
        metadata[_id] = {"title": "", "thumbnail": ""}

len(metadata)

  0%|          | 0/127 [00:00<?, ?it/s]

'NoneType' object has no attribute 'get'
arbbhHyRP90
'NoneType' object has no attribute 'get'
fpDaQxG5w4o
'NoneType' object has no attribute 'get'
vzBpSlexTVY
'NoneType' object has no attribute 'get'
bM9BXu9lPZ0
'NoneType' object has no attribute 'get'
xtOg44r6dsE


127

In [25]:
documents[21]

{'video_id': 'YqvhDPd1UEw',
 'text': " TRP o combined with generalized advanced estimation tae that's able to learn to master all their skills in this case that the robots learning to get up the reward is based on how close the head is to standing head height the closer the head of standing head height the higher the reward and then this was generalized to a much wider range of skills so what you see here is reinforced winning edge",
 'start_second': 535,
 'end_second': 562,
 'url': 'https://www.youtube.com/watch?v=YqvhDPd1UEw&t=535s'}

In [26]:
import json

with open("train.jsonl", "w") as f:
    for doc in documents:
        json.dump(doc, f)
        f.write('\n')

In [27]:
with open("train.jsonl") as f:
    d = f.readlines()

In [28]:
d[:3]

['{"video_id": "YqvhDPd1UEw", "text": " hi everyone welcome to lecture 12 of deep unsupervised learning spring 2020 today we\'ll cover representation learning in reinforcement learning first before I start I wanna give a big thank you to many colleagues and friends who have contributed to this lecture through sharing their insights illustrations slides and videos that all very directly contributed to well I\'ll be sharing in", "start_second": 0, "end_second": 28, "url": "https://www.youtube.com/watch?v=YqvhDPd1UEw&t=0s"}\n',
 '{"video_id": "YqvhDPd1UEw", "text": " this lecture here today thank you so Terry what class has been about unsupervised learning today we\'re actually going to look at how one surprise learning and reinforcement can be brought together in a way that inspires learning to make reinforced learning more efficient but we haven\'t covered reinforcement learning yet in this class and so what I\'m first gonna", "start_second": 28, "end_second": 52, "url": "https://www.yo

In [29]:
!pip install -U datasets sentence-transformers pinecone-client tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.0/214.0 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 50.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 60.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 77.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 2.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [30]:
from datasets import load_dataset

ytt = load_dataset(
    "pinecone/yt-transcriptions",
    split="train",
    revision="926a45"
)
ytt

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['video_id', 'text', 'start_second', 'end_second', 'url', 'title', 'thumbnail'],
    num_rows: 11298
})

In [31]:
for x in ytt:
    print(x)
    break

{'video_id': 'ZPewmEu7644', 'text': " hi this is Jeff Dean welcome to applications of deep neural networks of Washington University in this video we're going to look at how we can use ganz to generate additional training data for the latest on my a I course and projects click subscribe in the bell next to it to be notified of every new video Dan's have a wide array of uses beyond just the face generation that you", 'start_second': 0, 'end_second': 20, 'url': 'https://www.youtube.com/watch?v=ZPewmEu7644&t=0s', 'title': 'GANS for Semi-Supervised Learning in Keras (7.4)', 'thumbnail': 'https://i.ytimg.com/vi/ZPewmEu7644/maxresdefault.jpg'}


In [32]:
from sentence_transformers import SentenceTransformer
import torch

device = 'cuda' if torch.cuda.is_available() else 'cpu'

retriever = SentenceTransformer(
    'flax-sentence-embeddings/all_datasets_v3_mpnet-base'
)
retriever.to(device)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/9.85k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/591 [00:00<?, ?B/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: MPNetModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
)

In [33]:
embed_dim = retriever.get_sentence_embedding_dimension()
embed_dim

768

In [53]:
pip install pinecone-client

In [54]:
from pinecone import Pinecone

pc = Pinecone(api_key='6acb0685-3e9c-4128-8c17-df9997f70cf7')

In [58]:
from pinecone import Pinecone

pinecone = Pinecone(
    api_key="6acb0685-3e9c-4128-8c17-df9997f70cf7",
    environment="us-west1-gcp"
)

index = pinecone.Index("youtube-search")

In [59]:
from tqdm.auto import tqdm

docs = []  # this will store IDs, embeddings, and metadata

batch_size = 64

for i in tqdm(range(0, len(ytt), batch_size)):
    i_end = min(i+batch_size, len(ytt))
    # extract batch from YT transactions data
    batch = ytt[i:i_end]
    # encode batch of text
    embeds = retriever.encode(batch['text']).tolist()
    # each snippet needs a unique ID
    # we will merge video ID and start_seconds for this
    ids = [f"{x[0]}-{x[1]}" for x in zip(batch['video_id'], batch['start_second'])]
    # create metadata records
    meta = [{
        'video_id': x[0],
        'title': x[1],
        'text': x[2],
        'start_second': x[3],
        'end_second': x[4],
        'url': x[5],
        'thumbnail': x[6]
    } for x in zip(
        batch['video_id'],
        batch['title'],
        batch['text'],
        batch['start_second'],
        batch['end_second'],
        batch['url'],
        batch['thumbnail']
    )]
    # create list of (IDs, vectors, metadata) to upsert
    to_upsert = list(zip(ids, embeds, meta))
    # add to pinecone
    index.upsert(vectors=to_upsert)

  0%|          | 0/177 [00:00<?, ?it/s]

In [60]:
index.describe_index_stats()

{'dimension': 768,
 'index_fullness': 0.11298,
 'namespaces': {'': {'vector_count': 11298}},
 'total_vector_count': 11298}

In [63]:
query = "What is deep learning?"

xq = retriever.encode([query]).tolist()

In [67]:
xc = index.query(vector=xq, top_k=5,
                 include_metadata=True)

xc

{'matches': [{'id': 'Z6rxFNMGdn0-1154',
              'metadata': {'end_second': 1181.0,
                           'start_second': 1154.0,
                           'text': " terms of optimization but what's the "
                                   'algorithm for updating the parameters or '
                                   'updating whatever the state of the network '
                                   'is and then the the last part is the the '
                                   'data set like how do you actually '
                                   'represent the world as it comes into your '
                                   'machine learning system so I think of deep '
                                   'learning as telling us something about '
                                   'what does the model look like and '
                                   'basically to qualify as deep I',
                           'thumbnail': 'https://i.ytimg.com/vi/Z6rxFNMGdn0/maxresdefault.jpg'